# The Tokenizer Fun: From Bytes to BPE

A progressive, hands-on journey through the entire tokenization pipeline:  
**Raw string → Unicode code points → UTF-8 bytes → BPE tokens**

We'll build a tokenizer from scratch and then benchmark it against OpenAI's production tokenizer.

> **Heavily inspired by Andrej Karpathy's video: [Let's build the GPT Tokenizer](https://www.youtube.com/watch?v=zduSFxRajkE)**

In [ ]:
# Setup
import regex as re
import tiktoken
import collections
import json
import textwrap
import matplotlib.pyplot as plt
import numpy as np

---
## 1. The Unicode & Byte Inspection Lab

Every string in Python 3 is a sequence of **Unicode code points**.  
When we encode to UTF-8, each code point expands into **1–4 bytes**.

This is the foundation of tokenization: before any BPE, the model sees raw bytes.

In [86]:
def inspect_string(text):
    """Show the Unicode code points and UTF-8 byte breakdown for each character."""
    print(f"Input: {text!r}")
    print(f"{'Char':<6} {'Code Point':<12} {'UTF-8 Bytes':<20} {'# Bytes'}")
    print("─" * 50)
    
    total_bytes = 0
    for ch in text:
        utf8 = ch.encode('utf-8')
        total_bytes += len(utf8)
        hex_bytes = ' '.join(f'{b:02x}' for b in utf8)
        display_ch = ch if ch.isprintable() else repr(ch)
        print(f"{display_ch:<6} U+{ord(ch):04X}       {hex_bytes:<20} {len(utf8)}")
    
    print(f"\n→ {len(text)} characters = {total_bytes} UTF-8 bytes (expansion: {total_bytes/len(text):.2f}×)")

In [87]:
# Try different strings to see how byte counts vary!
inspect_string("Hello 🐍 世界!")

Input: 'Hello 🐍 世界!'
Char   Code Point   UTF-8 Bytes          # Bytes
──────────────────────────────────────────────────
H      U+0048       48                   1
e      U+0065       65                   1
l      U+006C       6c                   1
l      U+006C       6c                   1
o      U+006F       6f                   1
       U+0020       20                   1
🐍      U+1F40D       f0 9f 90 8d          4
       U+0020       20                   1
世      U+4E16       e4 b8 96             3
界      U+754C       e7 95 8c             3
!      U+0021       21                   1

→ 11 characters = 18 UTF-8 bytes (expansion: 1.64×)


In [88]:
# Emojis and non-Latin scripts use more bytes
inspect_string("مرحبا")

Input: 'مرحبا'
Char   Code Point   UTF-8 Bytes          # Bytes
──────────────────────────────────────────────────
م      U+0645       d9 85                2
ر      U+0631       d8 b1                2
ح      U+062D       d8 ad                2
ب      U+0628       d8 a8                2
ا      U+0627       d8 a7                2

→ 5 characters = 10 UTF-8 bytes (expansion: 2.00×)


In [89]:
# Flag emojis are TWO code points (regional indicators), 4 bytes each = 8 bytes per flag!
inspect_string("🇯🇵🇺🇸")

Input: '🇯🇵🇺🇸'
Char   Code Point   UTF-8 Bytes          # Bytes
──────────────────────────────────────────────────
🇯      U+1F1EF       f0 9f 87 af          4
🇵      U+1F1F5       f0 9f 87 b5          4
🇺      U+1F1FA       f0 9f 87 ba          4
🇸      U+1F1F8       f0 9f 87 b8          4

→ 4 characters = 16 UTF-8 bytes (expansion: 4.00×)


**Key takeaway:** A single visible character can be 1–8+ bytes. This is why raw byte encoding is inefficient and why we need BPE.

---
## 2. The BPE Algorithm Scratchpad

Byte Pair Encoding works by **iteratively merging the most frequent pair** of adjacent tokens into a new single token. This is how GPT-2/3/4, LLaMA, and other LLM tokenizers build their vocabulary.

### Core functions

In [ ]:
def get_stats(ids):
    """Count frequency of each adjacent pair in the token list."""
    counts = collections.Counter()
    for pair in zip(ids, ids[1:]):
        counts[pair] += 1
    return counts


def merge(ids, pair, new_id):
    """Replace every occurrence of `pair` in `ids` with `new_id`."""
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and (ids[i], ids[i + 1]) == pair:
            new_ids.append(new_id)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

### Training the tokenizer

Let's train on a small corpus and watch the merges happen step by step.

In [ ]:
training_text = (
    "The tokenizer splits text into tokens. Tokens are the atoms of language models. "
    "Language models predict the next token. The better the tokenizer, the better the model."
)

NUM_MERGES = 20

# Start from raw UTF-8 bytes
tokens = list(training_text.encode('utf-8'))
original_len = len(tokens)

# Build initial vocab: one entry per byte value (0–255)
vocab = {i: bytes([i]) for i in range(256)}
merges = {}  # (pair) -> new_id

# Track history for visualization
history = [{'step': 0, 'length': len(tokens)}]

print(f"Starting with {original_len} raw bytes\n")
print(f"{'Step':<6} {'Merged Pair':<25} {'New Token':<15} {'Freq':<6} {'Length':<8} {'Compression'}")
print("─" * 80)

for i in range(NUM_MERGES):
    stats = get_stats(tokens)
    if not stats:
        break
    top_pair = max(stats, key=stats.get)
    top_freq = stats[top_pair]
    new_id = 256 + i
    tokens = merge(tokens, top_pair, new_id)
    merges[top_pair] = new_id
    vocab[new_id] = vocab[top_pair[0]] + vocab[top_pair[1]]
    
    history.append({'step': i + 1, 'length': len(tokens)})
    
    p0 = vocab[top_pair[0]].decode('utf-8', errors='replace')
    p1 = vocab[top_pair[1]].decode('utf-8', errors='replace')
    merged = vocab[new_id].decode('utf-8', errors='replace')
    ratio = original_len / len(tokens)
    print(f"#{i+1:<5} {repr(p0) + ' + ' + repr(p1):<25} → {repr(merged):<13} {top_freq:<6} {len(tokens):<8} {ratio:.2f}×")

print(f"\n✅ Final: {original_len} bytes → {len(tokens)} tokens ({original_len/len(tokens):.2f}× compression)")

In [ ]:
# Visualize the compression over merge steps
steps = [h['step'] for h in history]
lengths = [h['length'] for h in history]
ratios = [original_len / l for l in lengths]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4.5))

ax1.plot(steps, lengths, 'o-', color='#7c3aed', markersize=5, linewidth=2)
ax1.fill_between(steps, lengths, alpha=0.1, color='#7c3aed')
ax1.set_xlabel('Merge Step')
ax1.set_ylabel('Sequence Length')
ax1.set_title('Sequence Length vs Merge Step')
ax1.grid(True, alpha=0.3)

ax2.bar(steps, ratios, color='#0ea5e9', edgecolor='#0284c7', alpha=0.8)
ax2.set_xlabel('Merge Step')
ax2.set_ylabel('Compression Ratio (×)')
ax2.set_title('Compression Ratio vs Merge Step')
ax2.grid(True, axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

**Key takeaway:** Each merge reduces the sequence length. With just 20 merges on a tiny corpus, we already get meaningful compression. Production tokenizers use ~100k merges on massive corpora.

---
## 3. Regex Splitter Visualization

Before running BPE, production tokenizers **pre-split** the input using regex patterns. This prevents "cross-category" merges — for example, we don't want `"dog."` to become a single token distinct from `"dog"`.

GPT-2 and GPT-4 use different splitting patterns:

In [ ]:
# The actual patterns used by GPT-2 and GPT-4 tokenizers
GPT2_PATTERN = r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""


def show_splits(text, pattern_name, pattern):
    """Show how a regex pattern splits the input text."""
    pieces = re.findall(pattern, text)
    print(f"{pattern_name} — {len(pieces)} pieces:")
    displayed = [repr(p) for p in pieces]
    print(f"  {' | '.join(displayed)}")
    print()

In [ ]:
test_text = """def hello_world():
    print("Hello, world!")
    return 42 + 3.14"""

print(f"Input: {test_text!r}\n")
show_splits(test_text, "GPT-2 Pattern", GPT2_PATTERN)
show_splits(test_text, "GPT-4 Pattern", GPT4_PATTERN)

In [ ]:
# Contractions — notice how the patterns handle apostrophes
contraction_text = "I'm don't they'll we've he'd she's"

print(f"Input: {contraction_text!r}\n")
show_splits(contraction_text, "GPT-2 Pattern", GPT2_PATTERN)
show_splits(contraction_text, "GPT-4 Pattern", GPT4_PATTERN)

In [ ]:
# Numbers — GPT-4 splits numbers into groups of 3 digits
number_text = "The price is $12345.67 and pi is 3.14159265"

print(f"Input: {number_text!r}\n")
show_splits(number_text, "GPT-2 Pattern", GPT2_PATTERN)
show_splits(number_text, "GPT-4 Pattern", GPT4_PATTERN)

**Key takeaway:** GPT-4's pattern splits numbers into 1–3 digit groups (helping with arithmetic), while GPT-2 keeps entire numbers together. Both patterns separate contractions from their base words.

---
## 4. The "Glitch Token" Investigator

The **"SolidGoldMagikarp" phenomenon**: some tokens exist in the vocabulary but were **never (or rarely) seen during model training**. When fed to the model, they can cause **bizarre hallucinations** because their embeddings are essentially random noise.

### How does this happen?

1. The **tokenizer** is trained on a large corpus (e.g., Common Crawl + Reddit)
2. Frequent substrings like Reddit usernames become single tokens
3. The **model** is then trained on a different (possibly filtered) dataset
4. If a token from step 2 never appears in step 3's data, its embedding is never updated
5. Result: the token exists but has a random, untrained embedding

In [ ]:
enc = tiktoken.get_encoding('cl100k_base')

# Compare "normal" strings vs potentially "glitchy" strings
test_strings = [
    # Potentially glitchy (long strings that compress into few tokens)
    'SolidGoldMagikarp',
    ' RandomRedditor',
    'cloneembedaliaboraliaboraliabor',
    'TheNitrome',
    'DragonMaworking',
    # Normal strings for comparison
    'Hello world',
    'The quick brown fox',
    'machine learning',
]

print(f"{'Input String':<35} {'Tokens':<8} {'Chars/Tok':<10} {'Token Breakdown'}")
print("─" * 90)

for s in test_strings:
    toks = enc.encode(s)
    chars_per_tok = len(s) / len(toks)
    pieces = [repr(enc.decode([t])) for t in toks]
    flag = " ⚠️" if chars_per_tok > 8 else ""
    print(f"{s:<35} {len(toks):<8} {chars_per_tok:<10.1f} {' '.join(pieces)}{flag}")

In [ ]:
# Visualize token frequency distribution from a sample corpus
sample_corpus = (
    "The quick brown fox jumps over the lazy dog. Machine learning models process text by first "
    "converting it into tokens. Each token is an integer that maps to a piece of text. Common words "
    "like the and is appear frequently, while rare technical terms appear infrequently. The "
    "tokenizer must handle all of these cases efficiently. Natural language processing has evolved "
    "significantly with the introduction of transformer-based architectures."
)

corpus_tokens = enc.encode(sample_corpus)
token_freq = collections.Counter(corpus_tokens)

# Top 20 tokens
top_tokens = token_freq.most_common(20)
labels = [repr(enc.decode([t])) for t, _ in top_tokens]
freqs = [f for _, f in top_tokens]

fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(range(len(labels)), freqs, color='#0ea5e9', edgecolor='#0284c7', alpha=0.8)
ax.set_yticks(range(len(labels)))
ax.set_yticklabels(labels, fontfamily='monospace', fontsize=9)
ax.invert_yaxis()
ax.set_xlabel('Frequency')
ax.set_title('Token Frequency Distribution (Top 20)')
ax.grid(True, axis='x', alpha=0.3)
for bar, freq in zip(ax.patches, freqs):
    ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height() / 2,
            str(freq), va='center', fontsize=9)
plt.tight_layout()
plt.show()

**Key takeaway:** Token frequency follows a power law. A few tokens (spaces, common words) dominate, while most tokens are rare. Glitch tokens are the extreme case: they exist in the vocab but have zero or near-zero frequency in the model's training data.

---
## 5. Tiktoken Benchmarking

Let's build an encoder/decoder from our BPE merges and compare against OpenAI's `cl100k_base` (used by GPT-4) across different content types.

In [ ]:
def bpe_encode(text, merges):
    """Encode text using our trained BPE merges."""
    tokens = list(text.encode('utf-8'))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        # Find the pair with the lowest merge index (earliest learned merge)
        pair = min(stats, key=lambda p: merges.get(p, float('inf')))
        if pair not in merges:
            break
        tokens = merge(tokens, pair, merges[pair])
    return tokens


def bpe_decode(ids, vocab):
    """Decode token ids back to a string."""
    raw = b''.join(vocab[idx] for idx in ids)
    return raw.decode('utf-8', errors='replace')


# Sanity check: round-trip encoding/decoding
test_str = "The tokenizer splits text into tokens."
encoded = bpe_encode(test_str, merges)
decoded = bpe_decode(encoded, vocab)
assert decoded == test_str, f"Round-trip failed: {decoded!r} != {test_str!r}"
print(f"✅ Round-trip passed: {test_str!r} → {len(encoded)} tokens → {decoded!r}")

In [ ]:
# Benchmark across content types
benchmarks = {
    'English Prose': (
        "The quick brown fox jumps over the lazy dog. "
        "Machine learning models process text by converting it into numerical tokens."
    ),
    'Python Code': textwrap.dedent("""\
        def fibonacci(n):
            if n <= 1:
                return n
            return fibonacci(n - 1) + fibonacci(n - 2)

        for i in range(10):
            print(f"fib({i}) = {fibonacci(i)}")
    """),
    'JSON': json.dumps({
        "name": "Alice", "age": 30, "scores": [95, 87, 92],
        "address": {"city": "Portland", "state": "OR"},
    }, indent=2),
    'YAML': textwrap.dedent("""\
        name: Alice
        age: 30
        scores: [95, 87, 92]
        address:
          city: Portland
          state: OR
    """),
    'Multilingual': "Hello world! こんにちは世界！مرحبا بالعالم! 🌍🚀",
    'Arithmetic': "1234 + 5678 = 6912, 9999 * 1001 = 10008999",
}

print(f"Our BPE: {NUM_MERGES} merges (vocab = {256 + NUM_MERGES}) vs GPT-4 cl100k_base (~100k merges)\n")
print(f"{'Content Type':<18} {'Chars':<7} {'Bytes':<7} {'Our BPE':<12} {'cl100k':<12} {'Ratio'}")
print("─" * 70)

our_totals, tik_totals, categories = [], [], []

for name, text in benchmarks.items():
    n_bytes = len(text.encode('utf-8'))
    our_n = len(bpe_encode(text, merges))
    tik_n = len(enc.encode(text))
    our_totals.append(our_n)
    tik_totals.append(tik_n)
    categories.append(name)
    
    winner = "←" if our_n <= tik_n else ""
    print(f"{name:<18} {len(text):<7} {n_bytes:<7} {our_n:<12} {tik_n:<12} {our_n/tik_n:.1f}× {winner}")

In [ ]:
# Side-by-side bar chart
fig, ax = plt.subplots(figsize=(12, 5))

x = np.arange(len(categories))
width = 0.35

ax.bar(x - width / 2, our_totals, width, label=f'Our BPE ({NUM_MERGES} merges)',
       color='#7c3aed', alpha=0.8)
ax.bar(x + width / 2, tik_totals, width, label='GPT-4 cl100k_base',
       color='#0ea5e9', alpha=0.8)

ax.set_xlabel('Content Type')
ax.set_ylabel('Token Count')
ax.set_title('Token Count: Our BPE vs GPT-4')
ax.set_xticks(x)
ax.set_xticklabels(categories, rotation=20, ha='right')
ax.legend()
ax.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Token-by-token comparison
compare_text = "Hello world! 🌍"

our_toks = bpe_encode(compare_text, merges)
tik_toks = enc.encode(compare_text)

print(f"Input: {compare_text!r}\n")

print(f"Our BPE ({len(our_toks)} tokens):")
for t in our_toks:
    decoded = vocab[t].decode('utf-8', errors='replace')
    print(f"  id={t:<5} → {decoded!r}")

print(f"\ncl100k_base ({len(tik_toks)} tokens):")
for t in tik_toks:
    decoded = enc.decode([t])
    print(f"  id={t:<5} → {decoded!r}")

---
## Learning Outcomes

By completing this notebook, you can now:

1. **Demystify Input Processing** — Explain the full pipeline: raw string → Unicode code points → UTF-8 bytes → integer tokens

2. **Master BPE** — Implement and debug Byte Pair Encoding, understanding how vocabulary size impacts sequence length and compute efficiency

3. **Diagnose LLM Artifacts** — Trace common LLM failures (bad arithmetic, can't reverse strings, "spelling" errors) back to tokenization boundaries

4. **Differentiate Training vs. Inference** — Distinguish between *training a tokenizer* (learning merge rules) and *encoding/decoding text* (applying those rules)

### Next steps

- Try training with more merges: set `NUM_MERGES = 1000`
- Use a larger training corpus
- Experiment with different regex split patterns
- Watch the full video: [Let's build the GPT Tokenizer — Andrej Karpathy](https://www.youtube.com/watch?v=zduSFxRajkE)